In [1]:
import spacy
import timeit
import math
import pandas as pd
import matplotlib.pyplot as plt
from os import path
from collections import Counter
from lxml import etree
from glob import glob
from unicodedata import normalize
from tqdm import tqdm

In [2]:
nlp = spacy.load('fr_core_news_lg')

In [3]:
nlp.add_pipe("sentencizer")

In [4]:
nlp.max_length = 4000000

In [5]:
def clean_text(txt):
    txt_res = normalize("NFKD", txt.replace('\xa0', ' '))
    txt_res = txt_res.replace('\\xa0', '')
    return txt_res

In [6]:
def pipeline_spacy_motifs(path):
    pos_ko = ["NUM", "X", "SYM", "PUNCT", "SPACE"]
    list_motif = []
    nombre_tokens = 0
    nombre_sentences = 0
    with open(path, encoding="utf8") as file:
        text = file.readlines()
        text_clean = clean_text(str(text).lower())
        docs = nlp(text_clean)
        nombre_tokens += len(docs)
        nombre_sentences += len(list(docs.sents))
        for token in docs:
            #si le token est bien un mot on récupère son lemme
            if token.pos_ not in pos_ko:
                if token.is_stop:
                    list_motif.append(token.pos_)
                else:
                    list_motif.append(token.lemma_)

    return list_motif, nombre_tokens, nombre_sentences

In [7]:
def bigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de bi-grammes"""
    list_bigram = []
    for indice_token in range(len(list_token)-1):
        bigram = list_token[indice_token]+'_'+list_token[indice_token+1]
        list_bigram.append(bigram)
    return list_bigram

In [8]:
def trigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de tri-grammes"""
    list_trigram = []
    for indice_token in range(len(list_token)-2):
        trigram = list_token[indice_token]+'_'+list_token[indice_token+1]+'_'+list_token[indice_token+2]
        list_trigram.append(trigram)
    return list_trigram

In [9]:
list_motif_select = ['DET',
 'PRON',
 'ADP',
 'ADV',
 'CCONJ',
 'SCONJ',
 'VERB',
 'AUX',
 'à',
 'ADJ',
 'éter',
 '-',
 'NOUN',
 'faire',
 'bien',
 '–',
 'même',
 'voir',
 'où',
 'être',
 'avoir',
 'vouloir',
 'il',
 'venir',
 'être',
 'petit',
 'homme',
 'savoir',
 'grand',
 'prendre',
 'là',
 'jour',
 'aller',
 'rien',
 'femme',
 'dire',
 'main',
 'jeune',
 'bon',
 'heure',
 'mettre',
 'été',
 'falloir',
 'temps',
 'non',
 'oeil',
 'après',
 'jamais',
 'fois',
 'croire',
 'ça',
 'pouvoir',
 'chose',
 'étaier',
 'fille',
 'passer',
 'tête',
 'devoir',
 'entendre',
 'coup',
 'porte',
 'trouver',
 '-t',
 'moment',
 'vie',
 'demander',
 'enfant',
 'lui',
 'mère',
 '-ce',
 'père',
 'oui',
 'voix',
 'donner',
 'trop',
 'point',
 '—',
 'monsieur',
 'vieux',
 'était',
 'attendre',
 'monde',
 'Monsieur',
 'air',
 'mot',
 'cœur',
 'déjà',
 'maison',
 'm.',
 'nuit',
 'vous',
 'jusqu’',
 'mort',
 'ici',
 'regarder',
 'sortir',
 'an',
 'long',
 'reprendre',
 'fort',
 'PROPN',
 'contre',
 'devenir',
 'tenir',
 'dieu',
 'chambre',
 'pauvre',
 'très',
 'pied',
 'ami',
 'revenir',
 'connaître',
 'regard',
 'bras',
 'soir',
 'nom',
 'sentir',
 'trè',
 'aucun',
 'bel',
 'noir',
 'perdre',
 'amour',
 'côté',
 'visage',
 'beaucoup',
 'comprendre',
 'rester',
 'eût',
 'mal',
 'presque',
 'rue',
 'voilà',
 'doute',
 'aimer',
 'mieux',
 'parler',
 'place',
 'penser',
 'instant',
 'peine',
 'laisser',
 'haut',
 'répondit',
 'blanc',
 'vrai',
 'chercher',
 'arriver',
 'loin',
 'appeler',
 'terre',
 'ouvrir',
 'partir',
 'sembler',
 'sourire',
 'matin',
 'pari',
 'je',
 'mourir',
 'porter',
 'paraître',
 'rire',
 'lettre',
 'fils',
 'parole',
 'nouveau',
 'plein',
 'bout',
 'eau',
 'entrer',
 'finir',
 'corps',
 'moi',
 'idée',
 'jeter',
 'fin',
 'asseoir',
 'beau',
 'table',
 'souvenir',
 'maître',
 'fond',
 'tour',
 'force',
 'lever',
 'dernier',
 'derrière',
 'gros',
 'autour',
 'près',
 'lieu',
 'rendre',
 'lire',
 'pourtant',
 'tomber',
 'raison',
 'sorte',
 'servir',
 'ailleurs',
 'silence',
 'marier',
 'oh',
 'affaire',
 'mme',
 'vivre',
 'INTJ',
 'esprit',
 'suivre',
 'milieu',
 'besoin',
 'année',
 'âme',
 'mois',
 'michel',
 'tard',
 'pensée',
 'bruit',
 'part',
 'ciel',
 'continuer',
 'famille',
 'écrier',
 'question',
 'droit',
 'face',
 'tendre',
 'route',
 'minute',
 'heureux',
 'apprendre',
 'ville',
 'montrer',
 'aime',
 'frère',
 'descendre',
 'premièr',
 'fenêtre',
 'aussitôt',
 'cheval',
 'bois',
 'lit',
 'feu',
 'suite',
 'tandis',
 'malgré',
 'rouge',
 'pièce',
 'soleil',
 'roi',
 'histoire',
 'fût',
 'comte',
 'chemin',
 'dame',
 'cher',
 'quitter',
 'bientôt',
 'sœur',
 'pierre',
 'geste',
 'passé',
 'garçon',
 'vite',
 'voiture',
 'œil',
 'murmurer',
 'mari',
 'monter',
 'épaule',
 'ordre',
 'pays',
 'mur',
 'autant',
 'agir',
 'jacques',
 'peur',
 'joie',
 'mauvais',
 'cri',
 'franc',
 'courir',
 'fou',
 'sang',
 'salle',
 'conduire',
 'tourner',
 'tirer',
 'mouvement',
 'ancien',
 'bonheur',
 'remettre',
 'marcher',
 'dè',
 'bouche',
 'plaisir',
 'permettre',
 'garde',
 'demande',
 'ajouter',
 'second',
 'cheveu',
 'travail',
 'compte',
 'joli',
 'lèvre',
 'doux',
 'songer',
 'lumière',
 'état',
 'froid',
 'cause',
 'fleur',
 'sûr',
 'rose',
 'hôtel',
 'vue',
 'partie',
 'prè',
 'reconnaître',
 'travers',
 'ombre',
 'cas',
 'retrouver',
 'grâce',
 'offrir',
 'argent',
 'mer',
 'rappeler',
 'docteur',
 'garder',
 'plutôt',
 'marquis',
 'crier',
 'figure',
 'jouer',
 'façon',
 'front',
 'doigt',
 'livre',
 'clair',
 'manger',
 'général',
 'essayer',
 'lorsqu’',
 'oreille',
 'bleu',
 'dernière',
 'larme',
 'moyen',
 'donné',
 'aujourd’hui',
 'pareil',
 'arbre',
 'anne',
 'souffrir',
 'êtes',
 'dormir',
 'bord',
 'dû',
 'oublier',
 'pousser',
 'papier',
 'lendemain',
 'saisir',
 'coin',
 'retourner',
 'jardin',
 'vraiment',
 'chef',
 'france',
 'saint',
 'sentiment',
 'profond',
 'gauche',
 'boire',
 'simple',
 'marche',
 'ensuite',
 'vif',
 'salon',
 'oncle',
 'revoir',
 'droite',
 'demain',
 'forme',
 'propre',
 'sens',
 'voisin',
 'terrible',
 'battre',
 'signe',
 'large',
 'secret',
 'êter',
 'journée',
 'au-dessus',
 'manière',
 'honneur',
 'craindre',
 'cour',
 'poursuivre',
 'vérité',
 'guerre',
 'sombre',
 'âge',
 'valoir',
 'soudain',
 'calme',
 'maman',
 'prier',
 'train',
 'chien',
 'vent',
 'midi',
 'trouvé',
 'malheureux',
 'trouve',
 'aprè',
 '-vou',
 'douleur',
 'rentrer',
 'duc',
 'ensemble',
 'bureau',
 'mariage',
 'verre',
 'semaine',
 'service',
 'triste',
 'parvenir',
 'attention',
 'court',
 'capitaine',
 'humain',
 'malheur',
 'promettre',
 'château',
 'retenir',
 'poser',
 'expliquer',
 'fortune',
 '*',
 'riche',
 'elizabeth',
 'pâle',
 'jambe',
 'désir',
 'robe',
 'impossible',
 'couleur',
 'disparaître',
 "qu\\'il",
 'anglais',
 'frapper',
 'voyage',
 'régi',
 'taire',
 'chevalier',
 'arrêter',
 'présence',
 'vide',
 'malade',
 'compter',
 "c\\'est",
 'cacher',
 'approcher',
 'ligne',
 'rêve',
 'hasard',
 'occuper',
 'soldat',
 'parent',
 'là.',
 'nature',
 'chère',
 'joue',
 'soin',
 'apercevoir',
 'poche',
 'atteindre',
 'endroit',
 'raconter',
 'donne',
 'jean',
 'village',
 'manquer',
 'effort',
 'gagner',
 'vert',
 'émotion',
 'on',
 'ennemi',
 'répondre',
 'nous',
 'colère',
 'visite',
 'nez',
 'écouter',
 'arme',
 'recevoir',
 'contraire',
 'escalier',
 'changer',
 'libre',
 'demeurer',
 'envoyer',
 'rencontrer',
 'quelqu’',
 'habitude',
 'genou',
 'françai',
 'scène',
 'foi',
 'intérieur',
 'grave',
 'ferme',
 'commencer',
 'peau',
 'choisir',
 'sujet',
 'trait',
 'amie',
 'entrée',
 'objet',
 'poitrine',
 'fer',
 'dos',
 'officier',
 'tranquille',
 'laisse',
 'bête',
 'pleurer',
 'envie',
 'glisser',
 'inconnu',
 'doucement',
 'laissé',
 'aimé',
 'don',
 'causer',
 'guère',
 'champ',
 'rejoindre',
 'conversation',
 'lourd',
 'charmant',
 'comtesse',
 'gris',
 'journal',
 'meilleur',
 'regarde',
 'remplir',
 'immense',
 'maîtresse',
 'surprendre',
 'étrange',
 'oser',
 'veille',
 'image',
 'mesure',
 'feuille',
 'travailler',
 'cousin',
 'église',
 'coucher',
 'médecin',
 'brave',
 'partout',
 'baron',
 'couvrir',
 'départ',
 'arrivé',
 'traverser',
 'arrière',
 'jeu',
 'exemple',
 'existence',
 'tuer',
 'courage',
 'seigneur',
 'surprise',
 'goût',
 'cours',
 'première',
 'intérêt',
 'vague',
 'réponse',
 'arrive',
 'police',
 'phrase',
 'demi',
 'payer',
 'serrer',
 'paraît',
 'fauteuil',
 'situation',
 'courant',
 'passion',
 'passage',
 'hier',
 'toucher',
 'foule',
 'appartenir',
 'lentement',
 'simplement',
 'préser',
 'pur',
 'brusquement',
 'propos',
 'impression',
 'campagne',
 'adresser',
 'sol',
 'connaître',
 'espèce',
 'beauté',
 'vin',
 'diable',
 'chapeau',
 'répéter',
 'arrivée',
 'apporter',
 'vivant',
 'davantage',
 'café',
 'dent',
 'nu',
 'œuvre',
 'prix',
 'détail',
 'espoir',
 'diriger',
 'faux',
 'chaise',
 'interrompre',
 'somme',
 'georges',
 'baiser',
 'confiance',
 'genre',
 'moindre',
 'arrêta',
 'fermer',
 'apparaître',
 'prince',
 'mademoisell',
 'dîner',
 'autrefois',
 'expression',
 'bas',
 'conscience',
 'produire',
 'crime',
 'tante',
 '�',
 'difficile',
 'naturel',
 'cou',
 'compagnie',
 'faute',
 'vivement',
 'vendre',
 'danger',
 'refuser',
 'type',
 'pont',
 'occasion',
 'fuir',
 'noble',
 'relever',
 'vaste',
 'premier',
 'volonté',
 'sauver',
 'étier',
 'jeunesse',
 'cent',
 'contenir',
 'langue',
 'taille',
 'oiseau',
 'joseph',
 'dur',
 'compagnon',
 'mademoiselle',
 'douter',
 'immobile',
 'échapper',
 'personne',
 'certainement',
 'auprès',
 'juge',
 'éclat',
 'neuf',
 'pain',
 'public',
 'tôt',
 'empêcher',
 'écrit',
 'musique',
 'mètre',
 'côte',
 'moitié',
 'siècle',
 'obtenir',
 'commissaire',
 'paix',
 'reçu',
 'but',
 'chaud',
 'preuve',
 'caractère',
 'jaune',
 'conseil',
 'pluie',
 'ressembler',
 'deviner',
 'accepter',
 'vieillard',
 'événement',
 'époque',
 'aider',
 'animal',
 'parti',
 'personnage',
 'domestique',
 'camarade',
 'direction',
 'carte',
 'rencontre',
 'fête',
 'remarquer',
 'mlle',
 'beal',
 'crainte',
 'inutile',
 'mémoire',
 'odeur',
 'commençer',
 'amoureux',
 'chanter',
 'projet',
 'interroger',
 'juger',
 'quartier',
 'appartement',
 'tombe',
 'déjeuner',
 'sommeil',
 'embrasser',
 'louis',
 'retirer',
 'curieux',
 'colonel',
 'parlé',
 'sauter',
 'demandé',
 'aperçut',
 'appuyer',
 'rouler',
 'facile',
 'cuisine',
 'soutenir',
 'pencher',
 'léger',
 'quelquefois',
 'faible',
 'nuage',
 'rapide',
 'rare',
 'patron',
 'miss',
 'sac',
 'entière',
 'trembler',
 'chance',
 'parfaitement',
 'plan',
 'emporter',
 'aventure',
 'baisser',
 'robert',
 'nombre',
 'abandonner',
 'sir',
 'espérer',
 '’',
 'morceau',
 'former',
 'loi',
 'accomplir',
 'capable',
 'armée',
 'avenir',
 'souffle',
 'circonstance',
 'suffire',
 'paysan',
 'liberté',
 'promener',
 'digne',
 'justice',
 'affreux',
 'forêt',
 'éloigner',
 'art',
 'île',
 'poste',
 'achever',
 'herbe',
 'amant',
 'présenter',
 'ignorer',
 'rapport',
 'véritabl',
 'horreur',
 'ordinaire',
 'soirée',
 'rôle',
 'parfait',
 'accompagner',
 'chapitre',
 'énorme',
 'distance',
 'assurer',
 'quart',
 'bande',
 'aide',
 'lampe',
 'absolument',
 'tromper',
 'victime',
 'mine',
 'palais',
 'allée',
 'marguerite',
 'pointe',
 'écrire',
 'promenade',
 'imaginer',
 'adresse',
 'connaissance',
 'peuple',
 'pitié',
 'hauteur',
 'tableau',
 'maigret',
 'charles',
 'chagrin',
 'condition',
 'intention',
 'paul',
 'rayon',
 'nicolas',
 'désespoir',
 'spectacle',
 'douceur',
 'avis',
 'remonter',
 'roger',
 'glace',
 'plaire',
 'engager',
 'chair',
 'théâtre',
 'page',
 'récit',
 'prise',
 'endormir',
 'brillant',
 'haine',
 'tort',
 'prison',
 'tendresse',
 'princesse',
 'montre',
 'prisonnier',
 'franchir',
 'écoute',
 'violent',
 'souffrance',
 'neige',
 'observer',
 'cabinet',
 'acte',
 'également',
 'voyageur',
 'billet',
 'rapidement',
 'à-dire',
 'trace',
 'réponder',
 'accent',
 'désormais',
 'puisqu’',
 'groupe',
 'titre',
 'vêtement',
 'étranger',
 'cesse',
 'témoin',
 'demeure',
 'position',
 'début',
 'toilette',
 'sourd',
 'adieu',
 'sonner',
 'drôle',
 'jeanne',
 'reconnaître',
 'honnête',
 'bouteille',
 'gorge',
 'cadavre',
 'course',
 'allemand',
 'secouer',
 'éprouver',
 'lueur',
 'commun',
 'plat',
 'pardon',
 'arracher',
 'voie',
 'lointain',
 'étage',
 'couper',
 'remercier',
 '-elle',
 'amitié',
 'sein',
 'angoisse',
 'boîte',
 'couvert',
 'branche',
 'léon',
 'pendre',
 'habit',
 'commença',
 'trou',
 'hiver',
 'costume',
 'folie',
 'bataille',
 'arrêté',
 'prêt',
 'livrer',
 'espère',
 'réalité',
 'honte',
 'ventre',
 'allé',
 'rivière',
 'cesser',
 'convenir',
 'inquiétude',
 'magnifique',
 'prière',
 'sérieux',
 'tantôt',
 'aspect',
 'auprè',
 'montagne',
 'dès',
 'décidé',
 'troupe',
 'pût',
 'société',
 'avouer',
 'étude',
 'action',
 'oublié',
 'là-bas',
 'flamme',
 'fatigue',
 'soulever',
 '«',
 'double',
 'commettre',
 'espace',
 'tristesse',
 'pardaillan',
 'politique',
 'dimanche',
 'sauvage',
 'complètement',
 'unique',
 'croix',
 'balle',
 'note',
 'peindre',
 'suzanne',
 'jusque',
 'cercle',
 'rideau',
 'tué',
 'angèle',
 'frais',
 'réflexion']

In [10]:
list_bigram_motif_select = ['ADP_DET',
 'PRON_PRON',
 'PRON_AUX',
 'SCONJ_PRON',
 'ADP_PRON',
 'PRON_VERB',
 'PRON_ADV',
 'à_DET',
 'CCONJ_PRON',
 'ADV_PRON',
 'CCONJ_DET',
 'ADV_ADP',
 'SCONJ_DET',
 'ADV_DET',
 'CCONJ_ADP',
 'VERB_DET',
 'PRON_DET',
 'PRON_éter',
 'ADV_VERB',
 'VERB_ADV',
 'PRON_ADP',
 'ADJ_DET',
 'ADV_ADV',
 'ADP_ADV',
 'AUX_ADV',
 'DET_ADJ',
 'ADV_AUX',
 'à_PRON',
 'DET_ADV',
 'VERB_ADP',
 'AUX_DET',
 'AUX_VERB',
 'ADV_SCONJ',
 'VERB_PRON',
 'PRON_faire',
 'DET_NOUN',
 'CCONJ_ADV',
 'CCONJ_SCONJ',
 'ADP_ADJ',
 'ADV_à',
 'PRON_avoir',
 'DET_petit',
 'ADP_ADP',
 'PRON_voir',
 'DET_main',
 'PRON_être',
 'où_PRON',
 'ADJ_PRON',
 'ADP_NOUN',
 'DET_homme',
 'DET_PRON',
 'VERB_SCONJ',
 'NOUN_ADP',
 'PRON_vouloir',
 'ADP_SCONJ',
 'SCONJ_ADP',
 'DET_grand',
 'faire_DET',
 'éter_DET',
 'PRON_CCONJ',
 'éter_ADV',
 'DET_femme',
 'DET_jeune',
 'PRON_falloir',
 'DET_oeil',
 'AUX_ADP',
 'DET_jour',
 'PRON_dire',
 'venir_ADP',
 'VERB_-',
 'DET_tête',
 'ADV_CCONJ',
 'DET_vie',
 'PRON_savoir',
 'AUX_été',
 'prendre_DET',
 'PRON_venir',
 '–_PRON',
 'DET_porte',
 '-_être',
 'SCONJ_SCONJ',
 'AUX_-ce',
 'ADV_éter',
 'PRON_aller',
 'DET_chose',
 'NOUN_PRON',
 'PRON_SCONJ',
 'ADJ_ADP',
 'PRON_mettre',
 'DET_mère',
 'ADP_VERB',
 'DET_heure',
 'DET_voix',
 'CCONJ_à',
 'DET_air',
 'PRON_croire',
 'DET_fois',
 'VERB_il',
 'PRON_trouver',
 'éter_ADP',
 'DET_père',
 'PRON_-',
 'DET_maison',
 'DET_temps',
 'PRON_prendre',
 'DET_enfant',
 '-t_il',
 'DET_moment',
 'DET_bon',
 'DET_nuit',
 'ADV_savoir',
 'DET_vieux',
 'PRON_devoir',
 'DET_mot',
 'voir_DET',
 'PRON_était',
 'coup_ADP',
 'PRON_donner',
 'DET_cœur',
 'à_ADV',
 '-_même',
 'VERB_à',
 'DET_bras',
 'ADP_être',
 'DET_chambre',
 'PRON_étaier',
 'après_DET',
 'DET_fille',
 'où_DET',
 'DET_regard',
 'homme_PRON',
 'ADP_même',
 'DET_même',
 'AUX_PRON',
 'main_ADP',
 'PRON_passer',
 'DET_monde',
 'DET_coup',
 'PRON_demander',
 'avoir_DET',
 '-_là',
 'DET_instant',
 'bien_SCONJ',
 'savoir_ADV',
 'ADV_ADJ',
 'DET_nom',
 'DET_pauvre',
 'DET_visage',
 'NOUN_DET',
 'contre_DET',
 'bien_PRON',
 'PRON_entendre',
 'PRON_pouvoir',
 'jusqu’_à',
 'ADP_doute',
 'vouloir_PRON',
 'faire_ADP',
 'DET_ami',
 '-_PRON',
 'DET_soir',
 'donner_DET',
 'PRON_sentir',
 'il_ADP',
 'mettre_ADP',
 'ADV_vouloir',
 'DET_long',
 'temps_ADP',
 'même_ADP',
 'DET_amour',
 'même_PRON',
 'PRON_attendre',
 'DET_mort',
 'jeune_fille',
 'mettre_à',
 'là_PRON',
 'heure_ADP',
 'DET_rue',
 'ADP_faire',
 'PRON_à',
 'passer_ADP',
 'PRON_regarder',
 'DET_ADP',
 'SCONJ_à',
 'autour_ADP',
 'PRON_tenir',
 'homme_ADP',
 'AUX_bien',
 'femme_PRON',
 'ADJ_AUX',
 'ADP_temps',
 'ADV_avoir',
 'vouloir_ADV',
 'PRON_aimer',
 'bien_DET',
 'ADV_pouvoir',
 'bout_ADP',
 'DET_fils',
 '–_DET',
 'jour_ADP',
 'prendre_ADP',
 'sortir_ADP',
 'AUX_à',
 'DET_place',
 '-ce_SCONJ',
 'CCONJ_VERB',
 'à_ADJ',
 'il_PRON',
 'DET_lettre',
 'rien_ADP',
 'PRON_écrier',
 'PRON_appeler',
 'croire_SCONJ',
 'VERB_CCONJ',
 'DET_idée',
 'DET_parole',
 'mettre_DET',
 'même_DET',
 'DET_premièr',
 'VERB_lui',
 'ça_PRON',
 'côté_ADP',
 '–_CCONJ',
 'tandis_SCONJ',
 'ADJ_CCONJ',
 'être_DET',
 'DET_table',
 'ADP_jour',
 'près_ADP',
 'porte_ADP',
 'avoir_ADV',
 'être_ADV',
 'savoir_SCONJ',
 'ADP_ailleurs',
 'milieu_ADP',
 'venir_PRON',
 'ADP_milieu',
 'INTJ_bien',
 'PRON_parler',
 'tête_ADP',
 'non_ADV',
 'AUX_jamais',
 'DET_bel',
 'ADJ_SCONJ',
 'passer_DET',
 'ADP_monde',
 'femme_ADP',
 'ADP_fond',
 'DET_dernier',
 '–_ADV',
 'à_coup',
 'il_ADV',
 'oeil_ADP',
 'PRON_eût',
 'DET_sorte',
 'ADP_grand',
 'voir_ADP',
 'PRON_connaître',
 'voir_ADV',
 'NOUN_ADV',
 'DET_tour',
 'CCONJ_ADJ',
 '-ce_ADV',
 'jour_PRON',
 'PRON_sembler',
 'là_ADP',
 'DET_corps',
 'fond_ADP',
 'PRON_lever',
 'entendre_DET',
 'savoir_PRON',
 'bien_ADP',
 'DET_esprit',
 'PRON_aime',
 'DET_pied',
 'DET_nouveau',
 'DET_eau',
 'point_ADP',
 'aller_PRON',
 'nom_ADP',
 'ADP_bout',
 'besoin_ADP',
 '—_CCONJ',
 'PRON_laisser',
 'chose_ADP',
 'DET_âme',
 'sorte_ADP',
 'pouvoir_PRON',
 'PRON_ouvrir',
 'non_PRON',
 'DET_pensée',
 'NOUN_CCONJ',
 'DET_fenêtre',
 'reprendre_DET',
 'PRON_ADJ',
 'oui_PRON',
 'DET_matin',
 'ADV_tard',
 'fois_ADP',
 'à_peine',
 'ouvrir_DET',
 'ADP_moment',
 'jeune_homme',
 'VERB_VERB',
 'DET_épaule',
 'PRON_revenir',
 'air_ADP',
 '-t_lui',
 'voir_PRON',
 'DET_terre',
 'là_DET',
 'VERB_rien',
 'CCONJ_CCONJ',
 'trouver_DET',
 'jusqu’_ADP',
 'ADV_bien',
 'pied_ADP',
 'DET_mari',
 'AUX_prendre',
 'DET_frère',
 'fille_ADP',
 '–_ADP',
 'DET_route',
 '—_ADP',
 'DET_affaire',
 'PRON_même',
 'SCONJ_ADJ',
 'VERB_être',
 'ADP_voir',
 'faire_ADV',
 'DET_DET',
 'ADP_an',
 'être_ADP',
 'PRON_montrer',
 'chose_PRON',
 'DET_bouche',
 'DET_force',
 'PRON_comprendre',
 'DET_lèvre',
 'DET_gros',
 'PRON_jeter',
 'après_PRON',
 'DET_ville',
 'DET_sœur',
 'DET_pièce',
 'ADV_voir',
 'DET_histoire',
 'DET_salle',
 'DET_geste',
 'ADV_faire',
 'fille_PRON',
 'ADP_heure',
 'DET_fin',
 'loin_ADP',
 'ADP_bon',
 'rien_ADV',
 'ADP_homme',
 'voir_SCONJ',
 'PRON_rester',
 'SCONJ_ADV',
 'ADP_-',
 'entrer_ADP',
 'PRON_rendre',
 'DET_famille',
 'être_DET',
 'ADP_pied',
 'ADP_aller',
 'être_ADP',
 'même_ADV',
 'plein_ADP',
 'derrière_DET',
 'être_ADV',
 'aller_ADP',
 'beaucoup_ADP',
 'ADP_suite',
 'maison_ADP',
 'finir_ADP',
 'PRON_porter',
 'à_NOUN',
 'temps_PRON',
 '-_NOUN',
 'anne_-',
 '-_marier',
 'dire_SCONJ',
 'enfant_PRON',
 '-_ADV',
 'DET_second',
 'lieu_ADP',
 'ADJ_ADV',
 'DET_cri',
 'lorsqu’_PRON',
 'lui_ADP',
 'ADV_être',
 'm._ADP',
 'devoir_PRON',
 'heure_PRON',
 'PRON_paraître',
 'DET_année',
 'PRON_penser',
 'DET_voiture',
 'PRON_arriver',
 'DET_maître',
 'ADP_petit',
 'revenir_ADP',
 'DET_bruit',
 'DET_lit',
 'mère_PRON',
 'DET_ombre',
 'DET_ordre',
 'rien_PRON',
 'DET_silence',
 'attendre_DET',
 'DET_ciel',
 'DET_oreille',
 'à_faire',
 'porter_DET',
 'DET_point',
 'DET_question',
 'DET_dernière',
 'AUX_ADJ',
 'VERB_bien',
 'malgré_DET',
 'ADV_grand',
 'ADP_bois',
 'moment_ADP',
 'asseoir_ADP',
 'PRON_agir',
 'lui_PRON',
 'PRON_chercher',
 'bien_ADV',
 'DET_cas',
 'DET_chemin',
 'NOUN_-',
 'PRON_sortir',
 'PRON_rappeler',
 'main_PRON',
 'regarder_DET',
 'partir_ADP',
 'DET_front',
 'PRON_servir',
 'fois_PRON',
 'DET_comte',
 'DET_côté',
 'avoir_VERB',
 'DET_mer',
 'falloir_SCONJ',
 'ADJ_à',
 'vie_ADP',
 'perdre_DET',
 'à_travers',
 'DET_hôtel',
 'DET_lendemain',
 'fois_DET',
 'CCONJ_bien',
 'place_ADP',
 'regarder_ADP',
 'au-dessus_ADP',
 'PRON_êtes',
 'PRON_asseoir',
 'rue_ADP',
 'à_pari',
 'DET_sourire',
 'falloir_PRON',
 'chercher_DET',
 'DET_minute',
 'AUX_il',
 'DET_pays',
 'AUX_-t',
 'demander_DET',
 'AUX_déjà',
 'faire_PRON',
 'jeter_DET',
 'DET_ancien',
 'ADP_pari',
 'tomber_ADP',
 'dire_DET',
 'trouver_ADP',
 'PRON_être',
 'jamais_PRON',
 'il_DET',
 'DET_cour',
 'DET_mouvement',
 'ADP_rien',
 'essayer_ADP',
 'fin_ADP',
 'PRON_êter',
 'DET_œil',
 'ADP_AUX',
 'à_VERB',
 'ADP_lieu',
 'bras_ADP',
 'DET_cheveu',
 'mort_ADP',
 'DET_mur',
 'ça_ADV',
 'mot_ADP',
 'père_PRON',
 'chambre_ADP',
 'PRON_demande',
 'DET_cheval',
 'DET_marquis',
 'DET_roi',
 'falloir_ADV',
 'droit_ADP',
 'ADP_face',
 'DET_figure',
 'long_ADP',
 'DET_lumière',
 'monde_PRON',
 'DET_état',
 'cœur_ADP',
 'DET_docteur',
 'ADP_où',
 'prè_ADP',
 'moment_où',
 'travers_DET',
 'DET_doigt',
 'parler_ADP',
 'bord_ADP',
 'Monsieur_DET',
 'ADP_ça',
 'dire_ADV',
 'DET_cher',
 'DET_beau',
 'même_temps',
 'ADP_CCONJ',
 'éter_PRON',
 'AUX_mettre',
 'VERB_vrai',
 'ADP_m.',
 'DET_journée',
 'cause_ADP',
 'porte_PRON',
 'PRON_trouve',
 'nuit_ADP',
 'CCONJ_non',
 'DET_droit',
 'DET_dame',
 'DET_bonheur',
 'DET_part',
 'jeune_femme',
 'DET_sentiment',
 'an_PRON',
 'PRON_souvenir',
 'oeil_PRON',
 'voix_ADP',
 'monsieur_ADP',
 'vie_PRON',
 'regard_ADP',
 'devenir_DET',
 'ADP_oeil',
 'appeler_DET',
 'DET_souvenir',
 'force_ADP',
 'tour_ADP',
 'DET_soleil',
 'ADV_rien',
 'dieu_PRON',
 'DET_dieu',
 'AUX_AUX',
 'servir_ADP',
 'ADP_cœur',
 'connaître_DET',
 'venir_à',
 'ADP_rire',
 'être_PRON',
 'PRON_suivre',
 'CCONJ_même',
 'ADP_eau',
 'PRON_retrouver',
 'bruit_ADP',
 'quitter_DET',
 'DET_mois',
 'mourir_ADP',
 'savoir_DET',
 'ADP_prendre',
 'ADP_dieu',
 'partie_ADP',
 'DET_joie',
 'PRON_remettre',
 'DET_façon',
 'VERB_ADJ',
 'coin_ADP',
 'PRON_–',
 'DET_poche',
 'souvenir_ADP',
 'enfant_ADP',
 'jour_DET',
 'éter_à',
 'croire_ADV',
 'visage_ADP',
 'idée_ADP',
 'DET_garçon',
 'ici_PRON',
 'face_ADP',
 'PRON_retourner',
 'ADP_côté',
 'jeter_ADP',
 'PRON_tourner',
 'DET_peine',
 'agir_ADP',
 'DET_raison',
 'DET_vrai',
 'PRON_devenir',
 'DET_jambe',
 'venir_DET',
 'PRON_conduire',
 'DET_coin',
 'PRON_fût',
 'DET_poitrine',
 'ADP_haut',
 'PRON_reprendre',
 'DET_feu',
 'DET_livre',
 'ADV_–',
 'ADP_terre',
 'mot_PRON',
 'SCONJ_ça',
 'PRON_perdre',
 'ADV_-',
 'DET_oncle',
 'avoir_vous',
 'AUX_là',
 'rien_à',
 'arriver_ADP',
 'ADP_femme',
 'dire_PRON',
 'AUX_donné',
 'ADP_jeune',
 'DET_endroit',
 'DET_travail',
 'DET_argent',
 'ADV_loin',
 'sentir_DET',
 'contre_PRON',
 'DET_escalier',
 'DET_âge',
 'DET_chef',
 'cœur_PRON',
 'DET_mauvais',
 'DET_nature',
 'DET_propre',
 'vouloir_VERB',
 'PRON_permettre',
 'AUX_vouloir',
 'main_DET',
 'DET_guerre',
 'PRON_apprendre',
 '-_PROPN',
 'PRON_occuper',
 'été_DET',
 'aller_ADV',
 'AUX_venir',
 'moment_PRON',
 'étaier_ADV',
 'PRON_prier',
 'attendre_ADP',
 'pouvoir_ADV',
 'ami_PRON',
 'pousser_DET',
 'à_côté',
 'moyen_ADP',
 'après_AUX',
 'tête_PRON',
 'descendre_ADP',
 'DET_sang',
 'PRON_taire',
 'ADV_trop',
 'comprendre_ADV',
 'DET_sens',
 'DET_haut',
 'jamais_DET',
 'DET_vérité',
 'NOUN_SCONJ',
 'AUX_dû',
 'ADP_fer',
 'premièr_fois',
 'DET_veille',
 'PRON_offrir',
 'DET_secret',
 'tendre_DET',
 'lettre_ADP',
 'DET_dos',
 'donner_ADP',
 'perdre_ADP',
 'ADV_être',
 'mort_PRON',
 'permettre_ADP',
 'même_SCONJ',
 'ADP_feu',
 'VERB_faire',
 'vous_ADV',
 'venir_ADV',
 'haut_ADP',
 'DET_jardin',
 'ADP_soleil',
 'lever_DET',
 'DET_présence',
 'DET_robe',
 'DET_arbre',
 'PRON_approcher',
 'DET_parent',
 'main_CCONJ',
 'derrière_PRON',
 'femme_CCONJ',
 'histoire_ADP',
 'voilà_DET',
 'DET_église',
 'ADP_france',
 'ça_VERB',
 'DET_honneur',
 'tenir_ADP',
 'ADJ_chose',
 'quelqu’_PRON',
 'amour_ADP',
 'PRON_donne',
 'ADV_fort',
 'fils_ADP',
 'soir_ADP',
 'Monsieur_PRON',
 'ADP_amour',
 'DET_manière',
 'AUX_très',
 'DET_comtesse',
 'ADP_loin',
 'DET_partie',
 'montrer_DET',
 'tenir_DET',
 'PRON_tirer',
 'étaier_ADP',
 'ADP_passer',
 '–_oui',
 'ADV_été',
 'ADP_savoir',
 'DET_fortune',
 'DET_conversation',
 'soir_PRON',
 'DET_chien',
 'DET_joli',
 'VERB_besoin',
 'DET_foule',
 'DET_ennemi',
 'DET_existence',
 'DET_situation',
 'connaître_ADV',
 'lire_DET',
 'DET_fleur',
 'DET_peau',
 'PROPN_-',
 'DET_capitaine',
 'garder_DET',
 'ADP_ciel',
 'DET_larme',
 'à_cause',
 'DET_chevalier',
 'DET_mariage',
 'homme_CCONJ',
 'fois_SCONJ',
 'nuit_PRON',
 'vivre_ADP',
 'an_ADP',
 'AUX_entendre',
 'DET_moyen',
 'DET_verre',
 'ADV_falloir',
 'ADP_contraire',
 'ADP_matin',
 'DET_être',
 '-_midi',
 'ADP_vieux',
 'suivre_DET',
 'ADV_comprendre',
 'ADV_venir',
 'part_ADP',
 'ADP_sang',
 'PRON_lire',
 '-ce_PRON',
 'DET_vent',
 'ADP_là',
 'table_ADP',
 'heure_CCONJ',
 'DET_bien',
 'ADP_bien',
 'DET_trait',
 'à_heure',
 'chemin_ADP',
 'maison_PRON',
 'bien_VERB',
 'monsieur_PRON',
 'jamais_VERB',
 'auprès_ADP',
 'PRON_adresser',
 'DET_douleur',
 'DET_bureau',
 'mère_ADP',
 'PRON_finir',
 'déjà_DET',
 'coup_PRON',
 'train_ADP',
 'raison_ADP',
 'PRON_arrêter',
 'ADP_chose',
 'heure_DET',
 'noir_ADP',
 'ADP_à',
 'PRON_quitter',
 'DET_papier',
 'ici_DET',
 'mouvement_ADP',
 'ADP_attendre',
 'instant_ADP',
 'DET_semaine',
 'DET_ligne',
 'SCONJ_VERB',
 'PRON_diriger',
 'voilà_PRON',
 'PRON_garder',
 'envie_ADP',
 'DET_chère',
 'DET_plaisir',
 'même_CCONJ',
 'temps_DET',
 'vouloir_SCONJ',
 'DET_scène',
 'été_ADP',
 'DET_moindre',
 '—_ADV',
 'DET_rêve',
 'DET_impression',
 'ADP_plein',
 'entendre_ADP',
 'DET_foi',
 'PRON_étier',
 'donné_DET',
 'moment_DET',
 'ADP_enfant',
 'monde_ADP',
 'cri_ADP',
 'grand_-',
 'monter_ADP',
 'ADV_entendre',
 'tête_CCONJ',
 'ADP_monsieur',
 'CCONJ_AUX',
 'DET_salon',
 'signe_ADP',
 'DET_maîtresse',
 'AUX_trop',
 'savoir_ADP',
 'DET_chaise',
 'ADV_croire',
 'DET_visite',
 'état_ADP',
 'PRON_reconnaître',
 'père_ADP',
 'éter_bien',
 'ADP_ici',
 'ADV_connaître',
 'là_ADV',
 'DET_signe',
 'PRON_envoyer',
 'vue_ADP',
 'jour_CCONJ',
 'tourner_ADP',
 'parler_ADV',
 'ADP_bord',
 'dire_ADP',
 'DET_espèce',
 'DET_vue',
 'DET_nez',
 'ADP_mois',
 'ici_ADP',
 'ordre_ADP',
 'PRON_partir',
 'DET_désir',
 'PRON_PROPN',
 'heureux_ADP',
 'DET_an',
 'sourire_ADP',
 'AUX_perdre',
 'corps_ADP',
 'ADP_roi',
 'penser_SCONJ',
 'pourtant_PRON',
 'marcher_ADP',
 'trouver_ADV',
 'PRON_entrer',
 'DET_mal',
 'DET_malheur',
 'DET_habitude',
 'DET_cousin',
 'PRON_apercevoir',
 'DET_image',
 'DET_entrée',
 'voix_PRON',
 'PRON_expliquer',
 'vouloir_vous',
 'DET_objet',
 'DET_journal',
 'non_ADP',
 'rien_VERB',
 'ADP_train',
 'matin_PRON',
 'déjà_ADP',
 'DET_forme',
 'ami_ADP',
 'suivre_ADP',
 'aimer_ADV',
 'PRON_cacher',
 'DET_garde',
 'ADP_hasard',
 'peur_ADP',
 '*_*',
 'mois_ADP',
 'DET_lieu',
 'blanc_ADP',
 'PRON_raconter',
 'vous_PRON',
 'dè_DET',
 'entendre_PRON',
 'ADP_œil',
 'répondit_DET',
 'ADP_main',
 'mieux_SCONJ',
 'DET_phrase',
 'AUX_passé',
 'parole_ADP',
 'ADV_aller',
 'trop_ADP',
 'ADP_argent',
 'jamais_ADP',
 'PRON_poser',
 'DET_réponse',
 'DET_bois',
 'chambre_PRON',
 'homme_DET',
 'présence_ADP',
 'DET_fauteuil',
 'ADP_point',
 'ADP_soir',
 'traverser_DET',
 '-_DET',
 'éter_ADJ',
 'ADP_travail',
 'ADP_joie',
 'DET_meilleur',
 'question_ADP',
 'DET_époque',
 'maître_ADP',
 'matin_ADP',
 'PRON_tendre',
 'ADV_étaier',
 'père_CCONJ',
 'PRON_ajouter',
 'rester_ADP',
 'saisir_DET',
 'DET_expression',
 'PRON_arrive',
 'ailleurs_PRON',
 'coup_DET',
 'DET_baron',
 'PRON_arrêta',
 'route_ADP',
 'ADP_autant',
 'ADP_venir',
 'tirer_ADP',
 'PRON_descendre',
 'peine_ADP',
 'entrée_ADP',
 'DET_beauté',
 'PRON_essayer',
 'salle_ADP',
 'devoir_être',
 'mère_CCONJ',
 'eût_VERB',
 'DET_arme',
 'DET_médecin',
 'chef_ADP',
 'ADV_devoir',
 'suite_ADP',
 'lui_ADV',
 'noir_PRON',
 'PRON_pencher',
 'tirer_DET',
 'VERB_–',
 'DET_feuille',
 'VERB_peur',
 'DET_première',
 'homme_ADV',
 'beaucoup_ADV',
 'DET_occasion',
 'comte_ADP',
 'revenir_à',
 'PRON_vivre',
 'aprè_-',
 'amour_PRON',
 'DET_intérieur',
 'DET_émotion',
 'ADP_château',
 'aller_à',
 'DET_officier']

In [11]:
list_trigram_motif_select = ['PRON_PRON_PRON',
 'PRON_PRON_AUX',
 'PRON_PRON_VERB',
 'SCONJ_PRON_PRON',
 'PRON_ADV_PRON',
 'ADP_PRON_PRON',
 'PRON_VERB_DET',
 'CCONJ_PRON_PRON',
 'SCONJ_PRON_ADV',
 'PRON_ADV_VERB',
 'CCONJ_ADP_DET',
 'ADV_ADP_DET',
 'PRON_AUX_VERB',
 'ADV_VERB_ADV',
 'PRON_AUX_DET',
 'PRON_ADV_AUX',
 'SCONJ_PRON_AUX',
 'PRON_VERB_ADV',
 'ADV_AUX_ADV',
 'PRON_VERB_ADP',
 'PRON_ADP_DET',
 'ADV_PRON_PRON',
 'CCONJ_PRON_ADV',
 'ADV_SCONJ_PRON',
 'PRON_AUX_ADV',
 'CCONJ_SCONJ_PRON',
 'ADV_PRON_VERB',
 'ADV_PRON_AUX',
 'ADP_PRON_ADP',
 'ADP_PRON_DET',
 'PRON_VERB_PRON',
 'PRON_PRON_ADV',
 'SCONJ_PRON_VERB',
 'ADP_ADJ_DET',
 'VERB_ADP_DET',
 'CCONJ_PRON_AUX',
 'VERB_-_être',
 'VERB_ADV_DET',
 'PRON_éter_DET',
 'ADP_SCONJ_PRON',
 'ADP_DET_ADJ',
 'DET_NOUN_ADP',
 'VERB_SCONJ_PRON',
 'PRON_PRON_éter',
 'ADP_DET_NOUN',
 'AUX_VERB_DET',
 'PRON_VERB_SCONJ',
 'CCONJ_PRON_VERB',
 'ADV_SCONJ_DET',
 'ADV_ADP_PRON',
 'ADV_à_DET',
 'PRON_AUX_ADP',
 'ADP_ADV_ADP',
 'ADP_PRON_CCONJ',
 'ADV_éter_ADV',
 'ADJ_PRON_PRON',
 'PRON_PRON_avoir',
 'SCONJ_SCONJ_PRON',
 'SCONJ_PRON_éter',
 'PRON_PRON_faire',
 'à_PRON_PRON',
 'PRON_venir_ADP',
 'ADP_PRON_VERB',
 'VERB_ADV_ADP',
 'PRON_ADV_savoir',
 'PRON_-_même',
 'ADP_ADP_DET',
 'NOUN_ADP_DET',
 'ADV_ADV_PRON',
 'SCONJ_ADP_DET',
 'ADP_DET_petit',
 'ADV_PRON_ADV',
 'PRON_faire_DET',
 'DET_main_ADP',
 'ADP_DET_main',
 'PRON_éter_ADP',
 'PRON_AUX_été',
 'PRON_SCONJ_PRON',
 'CCONJ_ADP_PRON',
 'ADP_DET_grand',
 'PRON_PRON_dire',
 'ADP_DET_vie',
 'où_PRON_PRON',
 'ADP_ADV_ADV',
 'CCONJ_ADV_PRON',
 'PRON_ADV_éter',
 'PRON_PRON_voir',
 'VERB_PRON_PRON',
 'DET_PRON_ADP',
 'ADP_DET_maison',
 'PRON_AUX_PRON',
 'ADP_DET_chambre',
 'VERB_ADP_PRON',
 'ADV_ADP_ADV',
 'PRON_CCONJ_PRON',
 'PRON_éter_ADV',
 'AUX_-ce_SCONJ',
 'DET_jeune_fille',
 'ADP_ADV_PRON',
 'AUX_ADV_DET',
 'PRON_avoir_DET',
 'ADP_DET_voix',
 'ADV_ADV_ADP',
 'ADJ_ADP_DET',
 'CCONJ_SCONJ_DET',
 'DET_homme_PRON',
 'DET_coup_ADP',
 'PRON_ADP_PRON',
 'PRON_mettre_à',
 'ADV_savoir_ADV',
 'PRON_PRON_vouloir',
 'ADP_DET_homme',
 'ADP_DET_air',
 'CCONJ_à_DET',
 'PRON_AUX_bien',
 'VERB_ADV_PRON',
 'ADP_DET_PRON',
 'AUX_ADP_DET',
 'DET_NOUN_PRON',
 'ADP_milieu_ADP',
 'à_DET_heure',
 'ADP_PRON_SCONJ',
 'ADP_DET_rue',
 'ADP_NOUN_ADP',
 '-ce_SCONJ_PRON',
 'ADP_PRON_ADV',
 'DET_porte_ADP',
 'PRON_donner_DET',
 'ADV_CCONJ_PRON',
 'ADP_PRON_faire',
 'PRON_PRON_trouver',
 'PRON_ADV_vouloir',
 'DET_tête_ADP',
 'PRON_ADV_avoir',
 'ADP_SCONJ_DET',
 'ADP_DET_femme',
 'ADP_DET_moment',
 'ADV_vouloir_ADV',
 'ADP_ADV_DET',
 'VERB_SCONJ_DET',
 'PRON_vouloir_PRON',
 'ADP_DET_ADV',
 'PRON_prendre_DET',
 'ADV_VERB_DET',
 'ADP_ADP_PRON',
 'PRON_PRON_falloir',
 'jusqu’_à_DET',
 'DET_ADV_ADP',
 'bien_SCONJ_PRON',
 'SCONJ_PRON_avoir',
 'ADP_NOUN_PRON',
 'DET_sorte_ADP',
 'AUX_VERB_PRON',
 'AUX_-ce_ADV',
 'PRON_PRON_être',
 'PRON_voir_DET',
 'PRON_VERB_à',
 'ADP_DET_jeune',
 'CCONJ_PRON_éter',
 'PRON_PRON_mettre',
 'ADP_DET_oeil',
 'SCONJ_PRON_DET',
 'DET_homme_ADP',
 'PRON_AUX_à',
 'ADP_DET_porte',
 'DET_femme_PRON',
 'SCONJ_PRON_vouloir',
 'AUX_ADV_ADV',
 'ADP_bout_ADP',
 'DET_air_ADP',
 'CCONJ_ADV_ADP',
 'main_ADP_DET',
 'ADP_DET_bras',
 'éter_ADP_DET',
 'PRON_AUX_-ce',
 'DET_oeil_ADP',
 'ADP_VERB_DET',
 'PRON_PRON_DET',
 'ADP_DET_nuit',
 'AUX_PRON_PRON',
 'ADV_VERB_PRON',
 'ADV_AUX_-ce',
 'croire_SCONJ_PRON',
 'DET_chose_ADP',
 'VERB_à_DET',
 'ADV_ADV_DET',
 'AUX_VERB_ADP',
 'ADP_fond_ADP',
 'DET_temps_ADP',
 'anne_-_marier',
 'PRON_mettre_ADP',
 'venir_ADP_PRON',
 'ADV_PRON_éter',
 'passer_ADP_DET',
 'CCONJ_PRON_DET',
 'ADV_à_PRON',
 'SCONJ_PRON_être',
 'AUX_ADV_VERB',
 'DET_femme_ADP',
 'ADP_DET_table',
 'PRON_PRON_venir',
 'PRON_ADV_pouvoir',
 'ADP_DET_mère',
 'CCONJ_ADV_DET',
 'ADP_DET_tête',
 'ADV_à_coup',
 'PRON_ADV_DET',
 'à_PRON_ADP',
 'PRON_CCONJ_DET',
 'ADP_PRON_-',
 'sortir_ADP_DET',
 'SCONJ_PRON_ADP',
 'ADV_AUX_jamais',
 'ADV_avoir_ADV',
 'NOUN_PRON_PRON',
 'ADP_DET_temps',
 'PRON_PRON_appeler',
 '–_PRON_PRON',
 'PRON_PRON_aller',
 'PRON_croire_SCONJ',
 'ADP_NOUN_DET',
 'ADJ_DET_monde',
 'DET_jour_ADP',
 'ADP_PRON_AUX',
 'ADV_AUX_VERB',
 'PRON_PRON_ADP',
 'ADP_DET_père',
 'DET_PRON_PRON',
 'savoir_SCONJ_PRON',
 'ADP_DET_cœur',
 'VERB_ADV_ADV',
 'ADP_DET_jour',
 'PRON_dire_SCONJ',
 'DET_heure_ADP',
 'DET_nom_ADP',
 'VERB_PRON_VERB',
 'éter_ADV_DET',
 'ADV_VERB_rien',
 'DET_jeune_homme',
 'DET_maison_ADP',
 'PRON_être_ADP',
 'AUX_ADV_ADP',
 'ADP_DET_même',
 'ADP_ADJ_PRON',
 'PRON_VERB_il',
 'bout_ADP_DET',
 'PRON_avoir_VERB',
 'DET_vie_ADP',
 'prendre_ADP_DET',
 'CCONJ_PRON_ADP',
 'PRON_PRON_prendre',
 'PRON_savoir_SCONJ',
 'ADP_DET_coup',
 'DET_main_PRON',
 'DET_ADJ_PRON',
 'DET_ADP_DET',
 'même_ADP_DET',
 'ADV_ADP_suite',
 'autour_ADP_PRON',
 'PRON_voir_ADP',
 'milieu_ADP_DET',
 'ADV_ADV_SCONJ',
 'PRON_PRON_était',
 'ADP_DET_salle',
 'DET_mère_PRON',
 'PRON_faire_ADP',
 'fond_ADP_DET',
 'faire_ADP_DET',
 'PRON_passer_ADP',
 'PRON_falloir_PRON',
 'PRON_PRON_croire',
 'ADJ_SCONJ_PRON',
 'PRON_à_DET',
 'PRON_falloir_SCONJ',
 'ADP_même_temps',
 'PRON_PRON_sentir',
 'PRON_être_DET',
 'à_travers_DET',
 'AUX_VERB_SCONJ',
 'ADP_DET_eau',
 'entrer_ADP_DET',
 'DET_jour_PRON',
 'DET_ADV_ADV',
 'DET_chose_PRON',
 'il_ADP_DET',
 'ADP_DET_enfant',
 'à_PRON_SCONJ',
 'PRON_voir_PRON',
 'PRON_PRON_donner',
 'à_ADJ_DET',
 'PRON_AUX_prendre',
 'ADP_DET_heure',
 'PRON_PRON_aime',
 'ADV_VERB_SCONJ',
 'VERB_ADV_SCONJ',
 'PRON_SCONJ_DET',
 'à_DET_porte',
 'ADP_faire_DET',
 'PRON_aller_PRON',
 'ADV_DET_ADJ',
 'ADP_DET_terre',
 'DET_bras_ADP',
 'ADP_DET_sorte',
 'PRON_PRON_aimer',
 'asseoir_ADP_DET',
 'PRON_AUX_ADJ',
 'PRON_trouver_ADP',
 '–_PRON_ADV',
 'PRON_PRON_tenir',
 'ADP_DET_cas',
 'DET_jeune_femme',
 'côté_ADP_DET',
 'PRON_VERB_vrai',
 'PRON_voir_ADV',
 'ADP_DET_amour',
 'DET_enfant_PRON',
 'ADP_PRON_voir',
 'falloir_SCONJ_PRON',
 'ADP_DET_vieux',
 'ADP_lieu_ADP',
 'DET_chambre_ADP',
 'SCONJ_à_DET',
 'ADP_DET_lit',
 'SCONJ_DET_ADJ',
 'à_DET_fois',
 'CCONJ_ADP_ADV',
 'ADV_être_ADV',
 'DET_vie_PRON',
 'tandis_SCONJ_DET',
 'DET_fois_ADP',
 'ADP_DET_premièr',
 'PRON_PRON_attendre',
 'PRON_PRON_agir',
 'ADP_DET_fenêtre',
 'PRON_VERB_bien',
 'VERB_il_PRON',
 'PRON_PRON_savoir',
 'VERB_il_ADP',
 'SCONJ_SCONJ_DET',
 'PRON_PRON_passer',
 'PRON_trouver_DET',
 'ADP_DET_ville',
 'CCONJ_DET_ADJ',
 'ADV_à_ADV',
 '-t_il_ADP',
 'porte_ADP_DET',
 'PRON_devoir_PRON',
 'PRON_ADV_être',
 'à_PRON_VERB',
 'DET_oeil_PRON',
 'PRON_prendre_ADP',
 'DET_porte_PRON',
 'ADV_VERB_ADP',
 'DET_voix_ADP',
 'DET_premièr_fois',
 'ADP_DET_côté',
 'dire_SCONJ_PRON',
 'PRON_ADV_voir',
 'ADP_moment_où',
 'DET_nuit_ADP',
 'PRON_VERB_VERB',
 'où_PRON_AUX',
 'PRON_PRON_demander',
 'ADP_DET_route',
 'DET_père_PRON',
 'DET_fille_ADP',
 'PRON_AUX_déjà',
 'PRON_VERB_CCONJ',
 'savoir_ADV_PRON',
 'à_DET_tour',
 'à_PRON_DET',
 'voir_ADP_DET',
 '-_là_PRON',
 'DET_mot_ADP',
 'SCONJ_ADJ_DET',
 'ADP_DET_bon',
 'ADV_PRON_faire',
 'ADP_DET_poche',
 'AUX_-t_il',
 'ADV_SCONJ_ADP',
 'PRON_éter_PRON',
 'ADP_DET_mort',
 'PRON_savoir_PRON',
 'tête_ADP_DET',
 'DET_rue_ADP',
 'ADV_à_NOUN',
 'à_DET_ADJ',
 '-_être_DET',
 'même_PRON_PRON',
 'PRON_être_ADV',
 'homme_PRON_PRON',
 'ADP_temps_ADP',
 'ADP_DET_monde',
 'ADV_CCONJ_ADV',
 'DET_mort_ADP',
 'PRON_regarder_ADP',
 'PRON_AUX_AUX',
 'DET_place_ADP',
 'VERB_DET_air',
 'SCONJ_ADP_PRON',
 'VERB_PRON_AUX',
 'DET_regard_ADP',
 'mettre_ADP_DET',
 'ADV_PRON_voir',
 'jour_ADP_DET',
 'autour_ADP_DET',
 'ADP_DET_âme',
 'DET_moment_ADP',
 'PRON_ADV_ADP',
 'DET_fin_ADP',
 'ADP_DET_épaule',
 'PRON_CCONJ_ADP',
 'PRON_AUX_mettre',
 'DET_idée_ADP',
 'là_PRON_PRON',
 'ADP_DET_fille',
 'PRON_PRON_rendre',
 'ADP_DET_esprit',
 'DET_long_ADP',
 'AUX_prendre_DET',
 'PRON_passer_DET',
 'DET_tour_ADP',
 '-_même_PRON',
 'DET_visage_ADP',
 'éter_ADV_ADP',
 'savoir_PRON_PRON',
 'ADP_pied_ADP',
 'VERB_ADV_à',
 'tomber_ADP_DET',
 'ADP_DET_visage',
 '–_PRON_AUX',
 'DET_ADJ_ADP',
 'à_côté_ADP',
 'PRON_faire_ADV',
 'oeil_ADP_DET',
 'ADP_DET_pièce',
 'pied_ADP_DET',
 'PRON_pouvoir_PRON',
 'ADV_PRON_avoir',
 'PRON_entendre_DET',
 'là_ADP_DET',
 'ADP_DET_regard',
 'ADV_DET_ADV',
 'ADP_DET_famille',
 'PRON_PRON_devoir',
 'PRON_PRON_connaître',
 'SCONJ_PRON_aller',
 'DET_main_CCONJ',
 'DET_tête_PRON',
 'AUX_VERB_-',
 'venir_ADP_DET',
 'SCONJ_PRON_falloir',
 'CCONJ_PRON_être',
 'DET_main_DET',
 'bien_PRON_PRON',
 'ADP_DET_cour',
 'ADP_face_ADP',
 'PRON_AUX_donné',
 'PRON_agir_ADP',
 'à_cause_ADP',
 'PRON_PRON_sembler',
 'PRON_mettre_DET',
 'DET_PRON_CCONJ',
 'CCONJ_DET_PRON',
 'DET_fois_PRON',
 'ADP_DET_instant',
 'DET_cœur_ADP',
 'DET_NOUN_DET',
 'PRON_parler_ADP',
 'DET_fois_DET',
 'PRON_PRON_demande',
 'AUX_ADV_PRON',
 'voir_SCONJ_PRON',
 '–_CCONJ_PRON',
 'PRON_ouvrir_DET',
 'DET_heure_PRON',
 'à_ADV_ADP',
 'VERB_CCONJ_PRON',
 'PRON_avoir_ADV',
 'CCONJ_ADJ_DET',
 'ADV_ADP_même',
 'CCONJ_DET_ADV',
 'PRON_PRON_entendre',
 'ADP_DET_ami',
 'PRON_AUX_jamais',
 'PRON_voir_SCONJ',
 'SCONJ_PRON_eût',
 'à_DET_NOUN',
 'DET_mot_PRON',
 'ADP_côté_ADP',
 'VERB_PRON_ADV',
 'ADV_CCONJ_DET',
 'DET_NOUN_CCONJ',
 'PRON_PRON_étaier',
 'DET_moment_PRON',
 'SCONJ_PRON_savoir',
 '-t_il_PRON',
 'ADP_DET_geste',
 'ADV_ADJ_DET',
 '-_être_ADV',
 'ADP_PRON_à',
 'PRON_jeter_ADP',
 'DET_ADV_DET',
 'DET_bruit_ADP',
 'ADP_DET_pays',
 'SCONJ_PRON_pouvoir',
 'PRON_savoir_ADV',
 'VERB_besoin_ADP',
 'DET_ADV_grand',
 'ADP_DET_long',
 'ADP_ADV_SCONJ',
 'heure_ADP_DET',
 'PRON_vouloir_VERB',
 'DET_instant_ADP',
 'à_DET_main',
 'ADV_ADV_ADV',
 'AUX_ADV_SCONJ',
 'à_DET_moment',
 'PRON_dire_DET',
 'ADP_DET_ADP',
 'PRON_PRON_laisser',
 'DET_lettre_ADP',
 'CCONJ_ADV_ADV',
 'près_ADP_DET',
 'ADP_ADV_VERB',
 'PRON_appeler_DET',
 'femme_PRON_PRON',
 'PRON_-_là',
 'AUX_VERB_ADV',
 'PRON_dire_PRON',
 'ADP_DET_mot',
 'ADP_DET_place',
 'DET_maison_PRON',
 'PRON_AUX_là',
 'bord_ADP_DET',
 'PRON_VERB_être',
 'au-dessus_ADP_DET',
 'PRON_AUX_vouloir',
 'VERB_DET_ADV',
 'SCONJ_DET_homme',
 'PRON_AUX_-t',
 'DET_nuit_PRON',
 'DET_monde_PRON',
 'DET_tête_CCONJ',
 'ADP_DET_coin',
 'VERB_DET_ADJ',
 'PRON_ADP_ADV',
 'SCONJ_PRON_venir',
 'homme_ADP_DET',
 'aprè_-_midi',
 'non_PRON_ADV',
 'DET_enfant_ADP',
 'ADP_DET_année',
 'DET_droit_ADP',
 'fois_ADP_DET',
 'VERB_ADP_ADV',
 'ADP_DET_hôtel',
 'ADP_VERB_ADP',
 'jeune_fille_PRON',
 'ADP_prendre_DET',
 'ADP_DET_mer',
 'ADV_ADV_VERB',
 'AUX_à_DET',
 'PRON_PRON_parler',
 'PRON_aller_ADP',
 'ADP_voir_DET',
 'revenir_ADP_DET',
 'cause_ADP_DET',
 'DET_cœur_PRON',
 'moment_où_PRON',
 'PRON_AUX_entendre',
 'PRON_ADV_falloir',
 'ADP_NOUN_CCONJ',
 'PRON_PRON_regarder',
 'DET_amour_ADP',
 'DET_moment_DET',
 'VERB_DET_chose',
 'CCONJ_à_PRON',
 'DET_pied_ADP',
 'ADP_DET_part',
 'ADP_DET_bouche',
 'DET_table_ADP',
 'à_DET_place',
 'lui_ADP_DET',
 'ADV_voir_ADV',
 'ADJ_PRON_AUX',
 'ADP_DET_ombre',
 'PRON_PRON_rappeler',
 'ADV_-_à-dire',
 'ADV_pouvoir_ADV',
 'PRON_sortir_ADP',
 '-_être_ADP',
 'éter_ADV_ADV',
 'ADP_bord_ADP',
 'DET_aprè_-',
 'INTJ_bien_PRON',
 'DET_ami_PRON',
 'AUX_-ce_PRON',
 'trouver_ADP_DET',
 'PRON_PRON_souvenir',
 'PRON_connaître_DET',
 'ADP_DET_façon',
 'DET_PRON_DET',
 'DET_temps_PRON',
 'tandis_SCONJ_PRON',
 'PRON_éter_à',
 'DET_histoire_ADP',
 'AUX_DET_ADV',
 'ADP_DET_lèvre',
 'DET_soir_ADP',
 'PRON_ADV_ADV',
 'CCONJ_PRON_faire',
 'à_DET_PRON',
 'ADP_DET_pauvre',
 'SCONJ_PRON_faire',
 'à_DET_fin',
 '-_être_PRON',
 'près_ADP_PRON',
 'ADP_DET_état',
 'ADP_DET_ciel',
 'PRON_dire_ADV',
 'ADV_ADV_à',
 'avoir_ADV_DET',
 'NOUN_PRON_AUX',
 'ADP_DET_bel',
 'DET_mère_ADP',
 'vouloir_ADV_PRON',
 'DET_fille_PRON',
 'DET_voix_PRON',
 'bien_SCONJ_DET',
 'CCONJ_CCONJ_PRON',
 'où_PRON_VERB',
 'PRON_AUX_très',
 'DET_force_ADP',
 'CCONJ_ADP_ADJ',
 'DET_point_ADP',
 'PRON_porter_DET',
 'VERB_lui_PRON',
 'ADP_ailleurs_PRON',
 'ADP_train_ADP',
 'à_PRON_faire',
 'PRON_asseoir_ADP',
 'coup_ADP_œil',
 'PRON_PRON_prier',
 'NOUN_PRON_ADV',
 'PRON_servir_ADP',
 'regarder_ADP_DET',
 'DET_cri_ADP',
 'coin_ADP_DET',
 'CCONJ_PRON_avoir',
 'ADP_DET_fils',
 'ADP_DET_corps',
 '—_ADP_DET',
 'DET_salle_ADP',
 'ADV_pouvoir_PRON',
 'AUX_ADP_PRON',
 'PRON_PRON_pouvoir',
 'PRON_savoir_bien',
 'PRON_venir_PRON',
 'à_coup_ADP',
 'PRON_demander_DET',
 'partie_ADP_DET',
 'jeter_ADP_DET',
 'DET_fils_ADP',
 'AUX_jamais_VERB',
 'jamais_PRON_ADV',
 'PRON_attendre_ADP',
 '-ce_PRON_PRON',
 'ADP_DET_chose',
 'ADP_DET_mari',
 'PRON_AUX_trop',
 'nom_ADP_DET',
 'ADP_DET_voiture',
 'place_ADP_DET',
 'être_ADP_DET',
 'CCONJ_DET_NOUN',
 'DET_ordre_ADP',
 'fois_SCONJ_PRON',
 'PRON_revenir_ADP',
 'ADJ_CCONJ_PRON',
 'SCONJ_PRON_devoir',
 'PRON_tourner_ADP',
 'SCONJ_DET_NOUN',
 'DET_coin_ADP',
 'DET_femme_CCONJ',
 'ADP_PRON_donner',
 'DET_jour_DET',
 'PRON_AUX_dû',
 'DET_père_ADP',
 'PRON_PRON_lever',
 'PRON_diriger_ADP',
 'DET_soir_PRON',
 'ADP_ADP_ADP',
 'ADP_DET_jardin',
 'PRON_vouloir_ADV',
 '-_même_DET',
 'CCONJ_ADV_à',
 'DET_mère_CCONJ',
 'DET_père_CCONJ',
 'ADV_CCONJ_ADP',
 'partir_ADP_DET',
 'ADV_PRON_savoir',
 'PRON_VERB_besoin',
 'ADJ_ADP_PRON',
 'ADV_falloir_ADV',
 'DET_espèce_ADP',
 'mettre_à_DET',
 'ADP_ADV_CCONJ',
 'femme_ADP_DET',
 'DET_chemin_ADP',
 'PRON_tenir_DET',
 'ADV_comprendre_ADV',
 'PRON_ADV_comprendre',
 'DET_ADV_PRON',
 'bien_ADP_DET',
 'PRON_attendre_DET',
 'être_ADP_DET',
 'DET_chambre_PRON',
 'jusqu’_à_PRON',
 'il_PRON_PRON',
 'PRON_éter_bien',
 'ADP_ADJ_ADP',
 'DET_oeil_DET',
 'PRON_PRON_trouve',
 '-_même_ADP',
 'DET_ombre_ADP',
 'PRON_VERB_ADJ',
 'ADP_DET_point',
 'PRON_montrer_DET',
 'DET_ADJ_CCONJ',
 '-_NOUN_-',
 'mort_ADP_DET',
 'PRON_essayer_ADP',
 'chose_PRON_PRON',
 'chose_ADP_DET',
 'fille_ADP_DET',
 'vouloir_SCONJ_PRON',
 'ADP_DET_pensée',
 'DET_entrée_ADP',
 'ADV_PRON_ADP',
 'AUX_donné_DET',
 'à_PRON_CCONJ',
 'DET_mouvement_ADP',
 'PRON_approcher_ADP',
 'ADP_DET_bureau',
 'ADP_DET_sourire',
 'DET_route_ADP',
 '-_PRON_PRON',
 'aller_ADP_DET',
 'ADP_DET_église',
 'PRON_tenir_ADP',
 'PRON_AUX_venir',
 '–_PRON_VERB',
 'ADP_DET_poitrine',
 'fin_ADP_DET',
 'ADV_ADV_-',
 'PRON_sentir_DET',
 'PRON_dire_ADP',
 'ADJ_DET_PRON',
 '-t_il_ADV',
 'ADP_DET_front',
 'jour_PRON_PRON',
 'PRON_ADV_connaître',
 'DET_corps_ADP',
 'AUX_été_DET',
 'DET_bout_ADP',
 'ADJ_à_DET',
 'ADP_DET_affaire',
 'temps_PRON_PRON',
 'PRON_vouloir_SCONJ',
 '—_CCONJ_PRON',
 'ADV_PRON_être',
 '–_ADV_PRON',
 'DET_parole_ADP',
 'NOUN_ADP_PRON',
 'DET_grand_-',
 'ADP_PRON_ADJ',
 'PRON_falloir_ADV',
 'à_DET_instant',
 'CCONJ_ADV_SCONJ',
 'NOUN_CCONJ_PRON',
 'DET_vie_CCONJ',
 'DET_part_ADP',
 'VERB_lui_ADP',
 'besoin_ADP_PRON',
 'PRON_finir_ADP',
 'DET_instant_PRON',
 'ADP_DET_nom',
 'PRON_croire_ADV',
 'ADP_où_PRON',
 'ADP_PRON_mettre',
 'ADP_DET_nouveau',
 'ADP_DET_propre',
 'ADJ_AUX_DET',
 'PRON_ADP_ADP',
 'long_ADP_DET',
 'PRON_ADJ_DET',
 'PRON_donner_ADP',
 'DET_oeil_CCONJ',
 'ADP_doute_DET',
 'DET_partie_ADP',
 'CCONJ_VERB_-',
 'temps_ADP_DET',
 'PRON_permettre_ADP',
 'AUX_à_PRON',
 'ADV_ADJ_ADP',
 'DET_état_ADP',
 'ADP_DET_chaise',
 'PRON_regarder_DET',
 'ADP_DET_salon',
 'ADJ_DET_jour',
 'CCONJ_PRON_mettre',
 'ADV_DET_temps',
 'DET_geste_ADP',
 'ADJ_PRON_ADV',
 'ADP_DET_nature',
 'ADP_DET_ancien',
 'PRON_PRON_servir',
 'ADP_DET_frère',
 'même_SCONJ_PRON',
 'SCONJ_à_PRON',
 'PRON_jeter_DET',
 'ADP_DET_escalier',
 'PRON_écrier_-t',
 'PRON_sembler_SCONJ',
 'PRON_AUX_croire',
 'bras_ADP_DET',
 'PRON_eût_VERB',
 'DET_esprit_ADP',
 'PRON_falloir_DET',
 'ADP_DET_fauteuil',
 'PRON_faire_PRON',
 'ADP_DET_pied',
 'ADV_besoin_ADP',
 'PRON_avoir_ADP',
 'PRON_falloir_ADP',
 'éter_ADP_PRON',
 'ADV_AUX_point',
 'DET_ami_ADP',
 'ADJ_SCONJ_DET',
 'perdre_ADP_DET',
 'CCONJ_PRON_voir',
 'oui_PRON_AUX',
 'CCONJ_PRON_falloir',
 'DET_tête_DET',
 'ADP_DET_journée',
 'DET_hôtel_ADP',
 'descendre_ADP_DET',
 'ADP_heure_ADP',
 'ADP_DET_mois',
 'voir_PRON_PRON',
 'CCONJ_PRON_savoir',
 'loin_ADP_DET',
 'ADV_DET_fois',
 'PRON_VERB_faire',
 'NOUN_ADV_PRON',
 'DET_amour_PRON',
 'PRON_AUX_perdre',
 'ADP_jour_ADP',
 'souvenir_ADP_DET',
 '–_ADP_DET',
 'DET_fois_SCONJ',
 'PRON_AUX_passé',
 'faire_ADP_PRON',
 'VERB_il_ADV',
 'à_DET_maison',
 'ADV_ADP_ADJ',
 'parler_ADP_DET',
 'PRON_trouver_ADV',
 'tenir_ADP_DET',
 'ADV_tard_PRON',
 'DET_porte_CCONJ',
 'ADP_PRON_–',
 'ADP_DET_sœur',
 'PRON_rappeler_DET',
 'savoir_SCONJ_DET',
 'PRON_écrier_DET',
 'DET_honneur_ADP',
 'ADV_PRON_croire',
 'PRON_vouloir_DET',
 'ADP_haut_ADP',
 'DET_pensée_ADP',
 'PRON_AUX_trouvé',
 'DET_ADJ_DET',
 'DET_vie_DET',
 'CCONJ_ADP_SCONJ',
 'agir_ADP_DET',
 'PRON_tendre_DET',
 'diriger_ADP_DET',
 'ADV_SCONJ_à',
 'haut_ADP_DET',
 'ADP_nom_ADP',
 'ADP_DET_œil',
 'PRON_savoir_DET',
 'finir_ADP_PRON',
 'PRON_PRON_donne',
 'donner_ADP_DET',
 'DET_jour_-',
 'ADV_ADP_temps',
 'SCONJ_DET_jeune',
 'PRON_VERB_peur',
 'ADV_ADJ_PRON',
 'VERB_rien_ADP',
 'ADP_jour_PRON',
 'ADV_SCONJ_SCONJ',
 'PRON_AUX_laissé',
 'où_PRON_ADV',
 'VERB_PRON_faire',
 'PRON_VERB_an',
 'SCONJ_DET_femme',
 'DET_bras_PRON',
 'DET_bras_CCONJ',
 'CCONJ_DET_femme',
 '-ce_ADV_PRON',
 'DET_ADJ_côté',
 'SCONJ_PRON_voir',
 'DET_parole_PRON',
 'PRON_lever_ADP',
 'DET_moyen_ADP',
 'VERB_à_PRON',
 'CCONJ_DET_petit',
 'ADP_monde_PRON',
 'ADP_DET_argent',
 'DET_pièce_ADP',
 'croire_SCONJ_DET',
 'PRON_VERB_rien',
 'VERB_ADJ_DET',
 'voilà_PRON_PRON',
 'ADP_doute_PRON',
 'ADP_DET_chemin',
 'temps_ADP_PRON',
 'SCONJ_PRON_dire',
 'ADP_doute_ADP',
 'PRON_arriver_ADP',
 'PRON_PRON_jeter',
 'DET_fenêtre_ADP',
 'PRON_demander_SCONJ',
 'ADP_DET_ordre',
 'DET_présence_ADP',
 'DET_mort_PRON',
 'ADV_DET_NOUN',
 'DET_monde_ADP',
 'PRON_AUX_apprendre',
 'ADP_ADV_à',
 'CCONJ_ADV_VERB',
 'PRON_demander_ADP',
 'ADV_VERB_il',
 'DET_petit_fille',
 'PRON_CCONJ_SCONJ',
 'air_ADP_DET',
 'ADP_DET_doigt',
 'ADP_-_NOUN',
 'PRON_aimer_ADV',
 'ADP_DET_force',
 'jeune_fille_ADP',
 'PRON_vouloir_bien',
 'ADP_DET_mur',
 'PRON_ADV_faire',
 'ADV_PRON_vouloir',
 'ADV_croire_ADV',
 'rien_ADP_DET',
 'DET_homme_ADV',
 'ADP_DET_lumière',
 'DET_affaire_ADP',
 'VERB_ADV_été',
 'ADP_DET_maître',
 'bruit_ADP_DET',
 'prè_ADP_DET',
 'VERB_DET_NOUN',
 'face_ADP_DET',
 'à_coup_DET',
 '-t_lui_ADP',
 'voir_SCONJ_DET',
 'NOUN_SCONJ_PRON',
 'vie_ADP_DET',
 'DET_âme_ADP',
 'ADP_DET_minute',
 'salle_à_manger',
 'DET_NOUN_ADV',
 'PRON_VERB_lui',
 'ADP_DET_forêt',
 'oui_PRON_PRON',
 '–_INTJ_bien',
 'SCONJ_PRON_fût',
 'ADP_an_PRON',
 'DET_eau_ADP',
 'ADV_étaier_ADV',
 'PRON_AUX_finir',
 'DET_vue_ADP',
 'ADP_coin_ADP',
 'DET_instant_DET',
 'à_ADV_prè',
 'ADP_DET_dernier',
 'PRON_pencher_ADP',
 'PRON_même_PRON',
 'PRON_occuper_ADP',
 'ADV_VERB_jamais',
 'DET_idée_PRON',
 'ADV_savoir_PRON',
 'ADV_DET_même']

In [12]:
list_motif_select

['DET',
 'PRON',
 'ADP',
 'ADV',
 'CCONJ',
 'SCONJ',
 'VERB',
 'AUX',
 'à',
 'ADJ',
 'éter',
 '-',
 'NOUN',
 'faire',
 'bien',
 '–',
 'même',
 'voir',
 'où',
 'être',
 'avoir',
 'vouloir',
 'il',
 'venir',
 'être',
 'petit',
 'homme',
 'savoir',
 'grand',
 'prendre',
 'là',
 'jour',
 'aller',
 'rien',
 'femme',
 'dire',
 'main',
 'jeune',
 'bon',
 'heure',
 'mettre',
 'été',
 'falloir',
 'temps',
 'non',
 'oeil',
 'après',
 'jamais',
 'fois',
 'croire',
 'ça',
 'pouvoir',
 'chose',
 'étaier',
 'fille',
 'passer',
 'tête',
 'devoir',
 'entendre',
 'coup',
 'porte',
 'trouver',
 '-t',
 'moment',
 'vie',
 'demander',
 'enfant',
 'lui',
 'mère',
 '-ce',
 'père',
 'oui',
 'voix',
 'donner',
 'trop',
 'point',
 '—',
 'monsieur',
 'vieux',
 'était',
 'attendre',
 'monde',
 'Monsieur',
 'air',
 'mot',
 'cœur',
 'déjà',
 'maison',
 'm.',
 'nuit',
 'vous',
 'jusqu’',
 'mort',
 'ici',
 'regarder',
 'sortir',
 'an',
 'long',
 'reprendre',
 'fort',
 'PROPN',
 'contre',
 'devenir',


In [13]:
list_bigram_motif_select

['ADP_DET',
 'PRON_PRON',
 'PRON_AUX',
 'SCONJ_PRON',
 'ADP_PRON',
 'PRON_VERB',
 'PRON_ADV',
 'à_DET',
 'CCONJ_PRON',
 'ADV_PRON',
 'CCONJ_DET',
 'ADV_ADP',
 'SCONJ_DET',
 'ADV_DET',
 'CCONJ_ADP',
 'VERB_DET',
 'PRON_DET',
 'PRON_éter',
 'ADV_VERB',
 'VERB_ADV',
 'PRON_ADP',
 'ADJ_DET',
 'ADV_ADV',
 'ADP_ADV',
 'AUX_ADV',
 'DET_ADJ',
 'ADV_AUX',
 'à_PRON',
 'DET_ADV',
 'VERB_ADP',
 'AUX_DET',
 'AUX_VERB',
 'ADV_SCONJ',
 'VERB_PRON',
 'PRON_faire',
 'DET_NOUN',
 'CCONJ_ADV',
 'CCONJ_SCONJ',
 'ADP_ADJ',
 'ADV_à',
 'PRON_avoir',
 'DET_petit',
 'ADP_ADP',
 'PRON_voir',
 'DET_main',
 'PRON_être',
 'où_PRON',
 'ADJ_PRON',
 'ADP_NOUN',
 'DET_homme',
 'DET_PRON',
 'VERB_SCONJ',
 'NOUN_ADP',
 'PRON_vouloir',
 'ADP_SCONJ',
 'SCONJ_ADP',
 'DET_grand',
 'faire_DET',
 'éter_DET',
 'PRON_CCONJ',
 'éter_ADV',
 'DET_femme',
 'DET_jeune',
 'PRON_falloir',
 'DET_oeil',
 'AUX_ADP',
 'DET_jour',
 'PRON_dire',
 'venir_ADP',
 'VERB_-',
 'DET_tête',
 'ADV_CCONJ',
 'DET_vie',
 'PRON_savoir',
 'AUX_é

In [14]:
list_trigram_motif_select

['PRON_PRON_PRON',
 'PRON_PRON_AUX',
 'PRON_PRON_VERB',
 'SCONJ_PRON_PRON',
 'PRON_ADV_PRON',
 'ADP_PRON_PRON',
 'PRON_VERB_DET',
 'CCONJ_PRON_PRON',
 'SCONJ_PRON_ADV',
 'PRON_ADV_VERB',
 'CCONJ_ADP_DET',
 'ADV_ADP_DET',
 'PRON_AUX_VERB',
 'ADV_VERB_ADV',
 'PRON_AUX_DET',
 'PRON_ADV_AUX',
 'SCONJ_PRON_AUX',
 'PRON_VERB_ADV',
 'ADV_AUX_ADV',
 'PRON_VERB_ADP',
 'PRON_ADP_DET',
 'ADV_PRON_PRON',
 'CCONJ_PRON_ADV',
 'ADV_SCONJ_PRON',
 'PRON_AUX_ADV',
 'CCONJ_SCONJ_PRON',
 'ADV_PRON_VERB',
 'ADV_PRON_AUX',
 'ADP_PRON_ADP',
 'ADP_PRON_DET',
 'PRON_VERB_PRON',
 'PRON_PRON_ADV',
 'SCONJ_PRON_VERB',
 'ADP_ADJ_DET',
 'VERB_ADP_DET',
 'CCONJ_PRON_AUX',
 'VERB_-_être',
 'VERB_ADV_DET',
 'PRON_éter_DET',
 'ADP_SCONJ_PRON',
 'ADP_DET_ADJ',
 'DET_NOUN_ADP',
 'VERB_SCONJ_PRON',
 'PRON_PRON_éter',
 'ADP_DET_NOUN',
 'AUX_VERB_DET',
 'PRON_VERB_SCONJ',
 'CCONJ_PRON_VERB',
 'ADV_SCONJ_DET',
 'ADV_ADP_PRON',
 'ADV_à_DET',
 'PRON_AUX_ADP',
 'ADP_ADV_ADP',
 'ADP_PRON_CCONJ',
 'ADV_éter_ADV',
 'ADJ_PRON_

In [15]:
def dict_freq_token(list_select, list_lemma):
    
    dict_result = dict.fromkeys(list_select)
    
    dict_temp = Counter(list_lemma)
        
    for key in dict_temp.keys():
        if key in dict_result.keys():
            dict_result[key] = dict_temp[key]/len(list_lemma)
    
    return dict_result

In [3]:
def moulinette(path_name, list_motif_select, list_bigram_motif_select, list_trigram_motif_select):
    
    i = 1
    nombre_total_tokens = 0
    nombre_total_sentences = 0
 
    dict_results = {}

    df_main = pd.DataFrame()

    
    print("\n\nBEGIN PROCESSING CORPUS-----------")
    
    for doc in tqdm(glob(path_name)):
        
        print("\n\nBEGIN PROCESSING NOVEL-----------")

            
        doc_name = path.splitext(path.basename(doc))[0]
        date = doc_name.split("_")[0]
        print(doc_name)
        
        #On recupere le texte des romans sous forme de listes de lemmes et de pos grâce à spacy
        
        list_motif_temp, nombre_tokens, nombre_sentences = pipeline_spacy_motifs(doc)
        
        print("PIPELINE SPACY ----------- OK")
        
        print("NOMBRE TOKENS = ", nombre_tokens)
        
        nombre_total_tokens += nombre_tokens
        nombre_total_sentences += nombre_sentences
        
        list_bigram_motif_temp = bigrammize(list_motif_temp)
        list_trigram_motif_temp = trigrammize(list_motif_temp)
    
        dict_results.update(dict_freq_token(list_motif_select, list_motif_temp))
        dict_results.update(dict_freq_token(list_bigram_motif_select, list_bigram_motif_temp))
        dict_results.update(dict_freq_token(list_trigram_motif_select, list_trigram_motif_temp))
         
        dict_results["index"] = doc_name
        
        df_temp = pd.DataFrame(dict_results, index=[0])
        
        df_main = df_main.append(df_temp, ignore_index = True)
        
        
        if i % 300 == 0:
            df_main.to_csv(r'output_mouli/motifs_3_'+str(i)+'.csv', index = True)
        
        i+=1
        
        print("END PROCESSING NOVEL --------------\n\n")
            
            #print("PROGRESSION ", round(nombre_total_tokens/2389336,2),'% COMPLETED\n')
    else:
        print(doc, 'DEJA VU')
        
    df_main.set_index("index", inplace = True)

    print("\n NOMBRE TOTAL TOKENS = ", nombre_total_tokens)
    print("\n NOMBRE TOTAL SENTENCES = ", nombre_total_sentences)

    print("\n RETURN LISTS RESULTS -----------")
    print("\n\n END PROCESSING CORPUS --------------\n\n")
            
    return df_main


In [2]:
#path_name = 'corpus_test/*.txt'
path_name = 'corpus_main_txt/*.txt'

In [ ]:
starttime = timeit.default_timer()
df_main = moulinette(path_name, list_motif_select, list_bigram_motif_select, list_trigram_motif_select, df_vue) 
print("Le temps total d'execution en secondes est de : ", timeit.default_timer() - starttime)



BEGIN PROCESSING CORPUS-----------


  0%|          | 0/2958 [00:00<?, ?it/s]



BEGIN PROCESSING NOVEL-----------
2003_Echenoz-Jean_Au-Piano
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  48753


  0%|          | 1/2958 [00:10<8:27:03, 10.29s/it]

END PROCESSING NOVEL --------------




BEGIN PROCESSING NOVEL-----------
1863_Feval-Paul_La-Fille-du-Juif-Errant
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  62340


...

END PROCESSING NOVEL --------------




BEGIN PROCESSING NOVEL-----------
1876_Brehat-Alfred-de_Le-bal-de-l-Opera
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  51098


 73%|███████▎  | 2149/2958 [20:53:17<9:26:16, 42.00s/it] 

END PROCESSING NOVEL --------------




BEGIN PROCESSING NOVEL-----------
2010_Gailly-Christian_Lily-et-Braine
PIPELINE SPACY ----------- OK
NOMBRE TOKENS =  42997


 73%|███████▎  | 2150/2958 [20:53:40<8:05:28, 36.05s/it]

END PROCESSING NOVEL --------------




BEGIN PROCESSING NOVEL-----------
1852_Dumas-Alexandre_La-comtesse-de-Charny


In [ ]:
starttime = timeit.default_timer()
df_main = moulinette(path_name, list_motif_select, list_bigram_motif_select, list_trigram_motif_select) 
print("Le temps total d'execution en secondes est de : ", timeit.default_timer() - starttime)

In [ ]:
df_main

In [52]:
df_main.to_csv(r'motifs.csv', index = True)